```
column_name type GENERATED { ALWAYS | BY DEFAULT } AS IDENTITY[ ( sequence_option ) ]
```

与 `SERIAL` 区别：
- 符合标准 SQL 规范
- 更具可读性和可维护性
- 具有可移植性和互操作性
- 更有利于数据完整性和维护，因为它提供了对属性进行控制的能力

In [17]:
import common.ipynb_importer
from db.pg.pg_00_common import *

cursor = pg_connect()

In [2]:
sql = """
CREATE TABLE color (
    color_id INT GENERATED ALWAYS AS IDENTITY,
    color_name VARCHAR NOT NULL
);

INSERT INTO color(color_name)
VALUES ('Red');
"""

cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x18dd6912c40>

In [3]:
sql = """
SELECT * FROM color;
"""

run_sql(cursor, sql)

   color_id color_name
0         1        Red


In [6]:
# 定义为 GENERATED ALWAYS 时，无法手动更新
sql = """
INSERT INTO color (color_id, color_name)
VALUES (2, 'Green');
"""

# 除非使用 OVERRIDING SYSTEM VALUE
sql2 = """
INSERT INTO color (color_id, color_name)
OVERRIDING SYSTEM VALUE 
VALUES(2, 'Green');
"""

# run_sql(cursor, sql)
run_sql(cursor, sql2)

UndefinedTable: 关系 "color" 不存在
LINE 2: INSERT INTO color (color_id, color_name)
                    ^

In [11]:
sql = """
DROP TABLE IF EXISTS color;

CREATE TABLE color (
    color_id INT GENERATED BY DEFAULT AS IDENTITY,
    color_name VARCHAR NOT NULL
);

INSERT INTO color (color_name)
VALUES ('White');

INSERT INTO color (color_id, color_name)
VALUES (2, 'Yellow');
"""
cursor.execute(sql)

sql = """
SELECT * from color;
"""

run_sql(cursor, sql)

   color_id color_name
0         1      White
1         2     Yellow


In [13]:
# 指定起始值和增长率
sql = """
DROP TABLE IF EXISTS color;

CREATE TABLE color (
    color_id INT GENERATED BY DEFAULT AS IDENTITY 
    (START WITH 10 INCREMENT BY 10),
    color_name VARCHAR NOT NULL
);

INSERT INTO color (color_name)
VALUES ('White');
"""
cursor.execute(sql)

sql = """
SELECT * from color;
"""

run_sql(cursor, sql)

   color_id color_name
0        10      White


In [14]:
sql = """
INSERT INTO color (color_name)
VALUES ('Purple');
"""
cursor.execute(sql)

sql = """
SELECT * from color;
"""

run_sql(cursor, sql)

   color_id color_name
0        10      White
1        20     Purple


将 Identity column 加入到已经存在的表中

In [18]:
sql = """
CREATE TABLE shape (
    shape_id INT NOT NULL,
    shape_name VARCHAR NOT NULL
);
"""
cursor.execute(sql)

# 增加
sql2 = """
ALTER TABLE shape 
ALTER COLUMN shape_id ADD GENERATED ALWAYS AS IDENTITY;
"""
cursor.execute(sql2)

# 修改
sql3 = """
ALTER TABLE shape
ALTER COLUMN shape_id SET GENERATED BY DEFAULT;
"""
cursor.execute(sql3)

# 删除
sql4 = """
ALTER TABLE shape
ALTER COLUMN shape_id
DROP IDENTITY IF EXISTS;
"""
cursor.execute(sql4)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x192b9b89b10>